In [1]:
import ants
import nilearn
from nilearn import image
import os, glob
import shutil
import nibabel as nib
import numpy as np
import pandas as pd

from nilearn.image import load_img
from nilearn.image import math_img
from nilearn.image import resample_to_img
from nilearn.image import smooth_img

from nipype.interfaces.ants import ApplyTransforms

In [21]:
def mni(directory):
    print(f'converting to mni for {i[:8]}')
    at = ApplyTransforms()
    at.inputs.dimension = 3
    at.inputs.input_image = f'/Volumes/Cingulate/Capsulotomy/France/segmentations/native/{i}'
    at.inputs.reference_image = '/Users/jp1590/Desktop/MNI152_T1_0.5mm.nii.gz'
    at.inputs.output_image = f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/{i[:-4]}_output.nii'
    at.inputs.transforms = f'/Volumes/Cingulate/Capsulotomy/France/FranceComplete/derivatives/leaddbs/{i[:8]}/normalization/transformations/{i[:8]}_from-anchorNative_to-MNI152NLin2009bAsym_desc-ants.nii.gz'
    at.inputs.invert_transform_flags = [False]
    at.run()

def binarize(directory):
    print(f'binarizing for {i[:8]}')
    lesion = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/{i[:-4]}_output.nii')
    mask = math_img('img > 0', img=lesion)
    mask.to_filename(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/{i[:8]}{i[15:-4]}_sim-binary_model-simbio.nii')

def z2(directory):
    print(f'creating zone 2 for {i[:8]}')
    if os.path.exists(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/{i[:8]}_zones1-2_sim-binary_model-simbio.nii'):
        z12 = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/{i[:8]}_zones1-2_sim-binary_model-simbio.nii')
        z1 = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/{i[:8]}_zone1_sim-binary_model-simbio.nii')
        z2 = math_img('img1-img2',img1=z1,img2=z12)
        z2_bin = math_img('img > 0', img=z2)
        nib.save(z2_bin,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/{i[:8]}_zone2_sim-binary_model-simbio.nii')

def resample(directory):
    print(f'resampling for {i[:8]}')
    mni = load_img('/Users/jp1590/Documents/MATLAB/leaddbs/templates/space/MNI152NLin2009bAsym/t1.nii')
    sub = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/{i[:8]}_zone1_sim-binary_model-simbio.nii')
    sub_re = resample_to_img(sub,mni,interpolation='nearest')
    if os.path.exists('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample'):
        print()
    else:
        os.mkdir('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample')
    nib.save(sub_re,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:8]}_zone1_sim-binary_model-simbio.nii')
    if os.path.exists(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/{i[:8]}_zone2_sim-binary_model-simbio.nii'):
        sub2 = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/{i[:8]}_zone2_sim-binary_model-simbio.nii')
        sub2_re = resample_to_img(sub2,mni,interpolation='nearest')
        nib.save(sub2_re,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:8]}_zone2_sim-binary_model-simbio.nii')
    if os.path.exists(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/{i[:8]}_zones1-2_sim-binary_model-simbio.nii'):
        sub12 = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/{i[:8]}_zones1-2_sim-binary_model-simbio.nii')
        sub12_re = resample_to_img(sub12,mni,interpolation='nearest')
        nib.save(sub12_re,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:8]}_zones1-2_sim-binary_model-simbio.nii')  

def copy(directory):
    print(f'copying {i[:8]} to be flipped')
    if os.path.exists('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip'):
        print()
    else:
        os.mkdir('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip')
    shutil.copyfile(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:8]}_zone1_sim-binary_model-simbio.nii',f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/{i[:8]}_zone1_sim-binary_model-simbio_flipped.nii')
    if os.path.exists(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:8]}_zone2_sim-binary_model-simbio.nii'):
        shutil.copyfile(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:8]}_zone2_sim-binary_model-simbio.nii',f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/{i[:8]}_zone2_sim-binary_model-simbio_flipped.nii')    
        
directory = [i for i in os.listdir('/Volumes/Cingulate/Capsulotomy/France/segmentations/native') if i.startswith('sub')]

for i in directory:
    mni(i)
    binarize(i)
    resample(i)
    z2(i)
    resample(i)
    copy(i)

converting to mni for sub-OCD1
binarizing for sub-OCD1
resampling for sub-OCD1
creating zone 2 for sub-OCD1
resampling for sub-OCD1

copying sub-OCD1 to be flipped
converting to mni for sub-OCD1
binarizing for sub-OCD1
resampling for sub-OCD1

creating zone 2 for sub-OCD1
resampling for sub-OCD1

copying sub-OCD1 to be flipped

converting to mni for sub-OCD2
binarizing for sub-OCD2
resampling for sub-OCD2

creating zone 2 for sub-OCD2
resampling for sub-OCD2

copying sub-OCD2 to be flipped

converting to mni for sub-OCD2
binarizing for sub-OCD2
resampling for sub-OCD2

creating zone 2 for sub-OCD2
resampling for sub-OCD2

copying sub-OCD2 to be flipped

converting to mni for sub-OCD3
binarizing for sub-OCD3
resampling for sub-OCD3

creating zone 2 for sub-OCD3
resampling for sub-OCD3

copying sub-OCD3 to be flipped

converting to mni for sub-OCD3
binarizing for sub-OCD3
resampling for sub-OCD3

creating zone 2 for sub-OCD3
resampling for sub-OCD3

copying sub-OCD3 to be flipped

conver

In [ ]:
#perform flipping in matlab

In [24]:
def resample(directory):
    print(f'resampling for {i[:8]}')
    mni = load_img('/Users/jp1590/Documents/MATLAB/leaddbs/templates/space/MNI152NLin2009bAsym/t1.nii')
    sub = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/{i[:8]}_zone1_sim-binary_model-simbio_flipped.nii')
    sub_re = resample_to_img(sub,mni,interpolation='nearest')
    nib.save(sub_re,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:8]}_zone1_sim-binary_model-simbio_flipped.nii')
    if os.path.exists(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/{i[:8]}_zone2_sim-binary_model-simbio_flipped.nii'):
        sub2 = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/{i[:8]}_zone2_sim-binary_model-simbio_flipped.nii')
        sub2_re = resample_to_img(sub2,mni,interpolation='nearest')
        nib.save(sub2_re,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:8]}_zone2_sim-binary_model-simbio_flipped.nii')
    
def add_flip(directory):
    print(f'adding flips together for {i[:8]}')
    if os.path.exists('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip'):
        print()
    else:
        os.mkdir('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip')
    z1_flip = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:9]}zone1_sim-binary_model-simbio_flipped.nii')
    z1 = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:9]}zone1_sim-binary_model-simbio.nii')
    z1_combine = math_img('img1+img2',img1=z1_flip,img2=z1)
    z1_bin = math_img('img > 0', img=z1_combine)
    nib.save(z1_bin,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/{i[:9]}zone1_sim-binary_model-simbio_flipped.nii')
    if os.path.exists(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:8]}_zone2_sim-binary_model-simbio_flipped.nii'):
        z2_flip = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:9]}zone2_sim-binary_model-simbio_flipped.nii')
        z2 = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/resample/{i[:9]}zone2_sim-binary_model-simbio.nii')
        z2_combine = math_img('img1+img2',img1=z2_flip,img2=z2)
        z2_bin = math_img('img > 0', img=z2_combine)
        nib.save(z2_bin,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/{i[:9]}zone2_sim-binary_model-simbio_flipped.nii')
        
def combine_z12(directory):
    print(f'combining zone1 and 2 together unweighted for {i[:8]}')
    if os.path.exists(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/{i[:9]}zone2_sim-binary_model-simbio_flipped.nii'):
        z1 = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/{i[:9]}zone1_sim-binary_model-simbio_flipped.nii')
        z2 = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/{i[:9]}zone2_sim-binary_model-simbio_flipped.nii')
        z12 = math_img('img1+img2',img1=z1,img2=z2)
        nib.save(z12,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/{i[:9]}zones1-2_sim-binary_model-simbio_flipped.nii')
        
def replace(path):
    print(f'adjusting weighting for {i[:8]}')
    if os.path.exists('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted'):
        print()
    else:
        os.mkdir('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted')
    if os.path.exists(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/{i[:9]}zones1-2_sim-binary_model-simbio_flipped.nii'):
        sub = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/{i[:9]}zones1-2_sim-binary_model-simbio_flipped.nii')
        sub_array = sub.get_fdata()
        new_data = np.where(sub_array == 2, 100, np.where(sub_array == 1, 50, sub_array))
        new_data = new_data.astype(np.int16)
        new_img = nib.Nifti1Image(new_data, sub.affine, sub.header)
        nib.save(new_img,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted/{i[:9]}zones1-2_sim-binary_model-simbio_flipped.nii')
    else:
        sub = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/sub-OCD5_zone1_sim-binary_model-simbio_flipped.nii')
        sub_array = sub.get_fdata()
        new_data = np.where(sub_array == 1, 50, sub_array)
        new_data = new_data.astype(np.int16)
        new_img = nib.Nifti1Image(new_data, sub.affine, sub.header)
        nib.save(new_img,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted/sub-OCD5_zone1_sim-binary_model-simbio_flipped.nii')
    
def smooth(directory):
    print(f'smoothing efields for {i[:8]}')
    if os.path.exists('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted/smooth'):
        print()
    else:
        os.mkdir('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted/smooth')
    if os.path.exists(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted/{i[:9]}zones1-2_sim-binary_model-simbio_flipped.nii'):
        sub = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted/{i[:9]}zones1-2_sim-binary_model-simbio_flipped.nii')
        gaussian = smooth_img(sub,fwhm=1.5) #change threshold here
        nib.save(gaussian,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted/smooth/{i[:9]}zones1-2_smooth_bilat_flip.nii')

def mask(directory):
    print(f'binarizing zones1-2 for {i[:8]}')
    if os.path.exists('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted/mask'):
        print()
    else:
        os.mkdir('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted/mask')
    if os.path.exists(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/{i[:9]}zones1-2_sim-binary_model-simbio_flipped.nii'):
        z12 = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/{i[:9]}zones1-2_sim-binary_model-simbio_flipped.nii')
        mask = math_img('img > 0', img=z12)
        if os.path.exists('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/z1-2_binarized'):
            print()
        else:
            os.mkdir('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/z1-2_binarized')
        mask.to_filename(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/z1-2_binarized/{i[:9]}zones1-2_sim-binary_model-simbio_flipped.nii')
    
    print(f'masking efields for {i[:8]}')
    if os.path.exists(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/z1-2_binarized/{i[:9]}zones1-2_sim-binary_model-simbio_flipped.nii'):
        bina = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/z1-2_binarized/{i[:9]}zones1-2_sim-binary_model-simbio_flipped.nii')
        efield = load_img(f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted/smooth/{i[:9]}zones1-2_smooth_bilat_flip.nii')
        mask = math_img('img1*img2',img1=bina,img2=efield)
        nib.save(mask,f'/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/weighted/mask/{i[:9]}zones1-2_smooth_bilat_flip.nii')
        
directory = [i for i in os.listdir('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip') if i.startswith('sub') and 'zone1' in i]

for i in directory:
    resample(i)
    add_flip(i)
    combine_z12(i)
    replace(i)
    smooth(i)
    mask(i)

resampling for sub-OCD1
adding flips together for sub-OCD1
combining zone1 and 2 together unweighted for sub-OCD1
adjusting weighting for sub-OCD1
smoothing efields for sub-OCD1
binarizing zones1-2 for sub-OCD1
masking efields for sub-OCD1
resampling for sub-OCD2
adding flips together for sub-OCD2

combining zone1 and 2 together unweighted for sub-OCD2
adjusting weighting for sub-OCD2

smoothing efields for sub-OCD2

binarizing zones1-2 for sub-OCD2


masking efields for sub-OCD2
resampling for sub-OCD3
adding flips together for sub-OCD3

combining zone1 and 2 together unweighted for sub-OCD3
adjusting weighting for sub-OCD3

smoothing efields for sub-OCD3

binarizing zones1-2 for sub-OCD3


masking efields for sub-OCD3
resampling for sub-OCD4
adding flips together for sub-OCD4

combining zone1 and 2 together unweighted for sub-OCD4
adjusting weighting for sub-OCD4

smoothing efields for sub-OCD4

binarizing zones1-2 for sub-OCD4


masking efields for sub-OCD4
resampling for sub-OCD5
a

In [ ]:
#last thing is to get the volumes for csv, outside of preprocessing

In [25]:
#load total volume
path = sorted(glob.glob(os.path.join('/Volumes/Cingulate/Capsulotomy/France/segmentations/mni/binarized_flip/all_binarize_flip/z1-2_binarized/','*.nii')))
vols = [image.load_img(i).get_fdata() for i in path]
ids = []

#get ids
for i in path:
    ids.append(i[104:112])

#get total volume of segmentations
total_vol = []
for i,v in enumerate(vols):
    total_vol.append(np.sum(v))
    
df = pd.DataFrame(
    {'ID': ids,
     'Total_Volume': total_vol
    })

df.to_csv('/Volumes/Cingulate/Capsulotomy/France/segmentations/France_Vol.csv')